# BDII -- Sesión 1 -- Procesado inicial de datos

Esta hoja muestra cómo procesar o curar un conjunto de datos para hacerlos más accesibles a la hora de introducirlos en bases de datos. Utilizaremos un conjunto de datos existente en Internet, que se descargará, se procesará y se convertirá en un formato universal como CSV o JSON. En particular se trabajará:

- La descarga de los datos.
- Inspección, identificación del formato y posible procesado.
- Generación de un formato fácilmente digerible por las BBDD, como CSV o JSON.

Comenzaremos instalando los paquetes necesarios:

In [1]:
!sudo apt-get update -qq

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [2]:
!sudo apt-get install -y p7zip tree

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip is already the newest version (16.02+dfsg-8).
p7zip set to manually installed.
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 22 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 0s (143 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselect

Importamos algunos paquetes estándar para la hoja

In [3]:
%pip install pandas matplotlib

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

%matplotlib inline
matplotlib.style.use('ggplot')

In [5]:
RunningInCOLAB = 'google.colab' in str(get_ipython()) if hasattr(__builtins__,'__IPYTHON__') else False

## Datos de Stackoverflow

El conjunto de datos de Stackoverflow es un *dump* de datos que cada cierto tiempo realiza el sitio web stackoverflow.com, en particular, la version en español, http://es.stackoverflow.com. El formato de los datos es XML, aunque es muy sencillo de extraer los datos, como veremos a continuación.

El contenido original se puede descargar directamente de los diferentes _dumps_ que se realizan de la página de archive.org: https://archive.org/details/stackexchange.

Sin embargo, nosotros descargaremos una versión fija previamente descargada para que todos partamos de los mismos datos.

## Descarga de los datos

En este caso los datos están disponibles en un repositorio git. Se pueden descargar también de la Web, pero se van actualizando. Los descargamos del repositorio git para que todos tengáis los mismos.

In [6]:
!wget https://github.com/dsevilla/bd2-data/raw/main/es.stackoverflow/es.stackoverflow.7z.001
!wget https://github.com/dsevilla/bd2-data/raw/main/es.stackoverflow/es.stackoverflow.7z.002
!wget https://github.com/dsevilla/bd2-data/raw/main/es.stackoverflow/es.stackoverflow.7z.003

--2025-01-30 10:30:32--  https://github.com/dsevilla/bd2-data/raw/main/es.stackoverflow/es.stackoverflow.7z.001
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dsevilla/bd2-data/main/es.stackoverflow/es.stackoverflow.7z.001 [following]
--2025-01-30 10:30:32--  https://raw.githubusercontent.com/dsevilla/bd2-data/main/es.stackoverflow/es.stackoverflow.7z.001
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 103809024 (99M) [application/octet-stream]
Saving to: ‘es.stackoverflow.7z.001’

es.stackoverflow.7z 100%[===================>]  99.00M   118MB/s    in 0.8s    

2025-01-30 10:30:33 (118 MB/s) - 

In [7]:
!ls -lh es.stackoverflow.7z*

-rw-r--r-- 1 root root  99M Jan 30 10:30 es.stackoverflow.7z.001
-rw-r--r-- 1 root root  99M Jan 30 10:30 es.stackoverflow.7z.002
-rw-r--r-- 1 root root 9.8M Jan 30 10:30 es.stackoverflow.7z.003


In [8]:
!7zr x es.stackoverflow.7z.001


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 103809024 bytes (99 MiB)

Extracting archive: es.stackoverflow.7z.001
  0% 1 Open           --
Path = es.stackoverflow.7z.001
Type = Split
Physical Size = 103809024
Volumes = 3
Total Physical Size = 217881776
----
Path = es.stackoverflow.7z
Size = 217881776
--
Path = es.stackoverflow.7z
Type = 7z
Physical Size = 217881776
Headers Size = 244
Method = LZMA2:24
Solid = +
Blocks = 1

  0%      0% - Comments.xml                     1% - Comments.xml                     2% - Comments.xml                     3% - Comments.xml                   

In [9]:
!ls -lh *.xml

-rw-r--r-- 1 root root 206M Dec  4  2023 Comments.xml
-rw-r--r-- 1 root root 983M Dec  4  2023 Posts.xml
-rw-r--r-- 1 root root 223K Dec  4  2023 Tags.xml
-rw-r--r-- 1 root root  73M Dec  4  2023 Users.xml
-rw-r--r-- 1 root root  70M Dec  4  2023 Votes.xml


## Inspección y procesado

Podemos inspeccionar los ficheros `.xml` para ver su contenido. Son XML, sí, pero ¿con qué formato?

In [13]:
!head Posts.xml

﻿<?xml version="1.0" encoding="utf-8"?>
<posts>
  <row Id="1" PostTypeId="1" AcceptedAnswerId="2" CreationDate="2015-10-29T15:56:52.933" Score="40" ViewCount="780" Body="&lt;p&gt;Estoy creando un servicio usando &lt;em&gt;ASP.NET WebApi&lt;/em&gt;. Quiero añadir soporte para la negociación del tipo de contenido basado en extensiones en el &lt;em&gt;URI&lt;/em&gt;, así que he añadido lo siguiente al código de inicialización del servicio:&lt;/p&gt;&#xA;&#xA;&lt;pre&gt;&lt;code&gt;public static class WebApiConfig&#xA;{&#xA;  public static void Register(HttpConfiguration config)&#xA;  {&#xA;    config.Formatters.JsonFormatter.AddUriPathExtensionMapping(&quot;json&quot;, &quot;application/json&quot;);&#xA;    config.Formatters.XmlFormatter.AddUriPathExtensionMapping(&quot;xml&quot;, &quot;application/xml&quot;);&#xA;  }&#xA;}&#xA;&lt;/code&gt;&lt;/pre&gt;&#xA;&#xA;&lt;p&gt;Para que esto funcione necesito crear dos rutas para cada acción del controlador (estoy usando exclusivamente enrutamie

In [14]:
!tail Posts.xml

  <row Id="609176" PostTypeId="1" CreationDate="2023-12-03T00:44:57.027" Score="0" ViewCount="11" Body="&lt;p&gt;¿Cada proyecto creado en gitlab tiene que tener su propia clave SSH o es una clave SSH para todos los proyectos creados?&lt;/p&gt;&#xA;" OwnerUserId="345626" LastActivityDate="2023-12-03T00:44:57.027" Title="Cuenta SSH en Gitlab" Tags="&lt;ssh&gt;&lt;gitlab&gt;" AnswerCount="0" CommentCount="0" ContentLicense="CC BY-SA 4.0" />
  <row Id="609177" PostTypeId="1" CreationDate="2023-12-03T00:47:10.527" Score="0" ViewCount="5" Body="&lt;p&gt;Estoy intentando esto.&lt;/p&gt;&#xA;&lt;pre&gt;&lt;code&gt;val str= imgUri.toString()//aqui convierto el archivo de uri component a un tipo String&#xA;&#xA;str.toUri()// aqui regreso la cadena a Uri.&#xA;&#xA;dataSet.add(datacomida(str1,str))// agrego a un array tipo mutablelist&amp;lt;datacomida&amp;gt;=arrayListof()&#xA;str // es de tipo Uri supuestamente ya que la converti.&#xA;str1// es de tipo String&#xA;&lt;/code&gt;&lt;/pre&gt;&#xA;&l

Aunque se puede procesar el formato XML, lo que podemos ver es que cada entrada es exactamente una línea que comienza por "`<row`", y que contiene un conjunto de atributos en formato "`atributo="valor"`". Si lo comprobamos, incluso no existirá ninguna comilla doble **dentro** de otra comilla doble, así que podemos extraer esos pares de forma facil.

La siguiente función procesa el fichero XML línea a línea. Primero separa la parte inicial "`<row`", y después procesa cada par clave/valor. Lo único que hace es construir el conjunto de atributos que hay en todas las entradas. Como vimos, cada fila contenía atributos diferentes. Queremos obtenerlos todos.

La función, en vez de retornar una lista, que ocuparía mucha memoria, retorna un generador, que es una lista (de pares clave-valor, un diccionario) que se va generando a medida que se recorre. Por eso utiliza la construcción `yield` de Python. Esto hace que la función se detenga, y cuando se le pide el siguiente elemento, continúa desde donde se quedó (corrutina).

In [15]:
import re
from collections.abc import Iterator

def generate_elements_from_lines(filename: str) -> Iterator[dict[str, str]]:

  def get_attrs(line: str) -> dict[str, str]:
    (_, attrs) = line.split("<row ", 2)
    return {m.group(1): m.group(2)
              for m in re.finditer(r"(\w+?)=\"(.*?)\"", attrs)}

  with open(filename, "r") as f:
    for line in f:
      if "<row" in line:
        yield get_attrs(line)

In [16]:
first_row: dict[str, str] = next(generate_elements_from_lines("Posts.xml"))

In [17]:
first_row.keys()

dict_keys(['Id', 'PostTypeId', 'AcceptedAnswerId', 'CreationDate', 'Score', 'ViewCount', 'Body', 'OwnerUserId', 'LastEditorDisplayName', 'LastEditDate', 'LastActivityDate', 'Title', 'Tags', 'AnswerCount', 'CommentCount', 'ContentLicense'])

Hay que extraer el conjunto de atributos para saber qué columnas tendrá nuestra tabla/CSV o archivo JSON. Recuérdese que las dos primeras filas del archivo XML tenían diferentes atributos. ¿Cómo se haría esto?

In [18]:
from collections.abc import Iterator

def get_all_attrs(iterator: Iterator[dict[str,str]]) -> set[str]:
  all_attrs: set[str] = set()
  for row in iterator:
    all_attrs.update(row.keys())
  return all_attrs

all_attrs: set[str] = get_all_attrs(generate_elements_from_lines("Posts.xml"))

El conjunto de atributos es pues:

In [19]:
all_attrs

{'AcceptedAnswerId',
 'AnswerCount',
 'Body',
 'ClosedDate',
 'CommentCount',
 'CommunityOwnedDate',
 'ContentLicense',
 'CreationDate',
 'FavoriteCount',
 'Id',
 'LastActivityDate',
 'LastEditDate',
 'LastEditorDisplayName',
 'LastEditorUserId',
 'OwnerDisplayName',
 'OwnerUserId',
 'ParentId',
 'PostTypeId',
 'Score',
 'Tags',
 'Title',
 'ViewCount'}

Como sabemos que el atributo `Id` va a ser la clave primaria, lo ponemos al principio. Además, generamos una lista, no un conjunto, para que el orden sea conocido.

In [20]:
all_attrs.remove('Id')
all_attrs = list(all_attrs)
all_attrs.insert(0,'Id')
all_attrs

['Id',
 'Score',
 'CreationDate',
 'AnswerCount',
 'ParentId',
 'ClosedDate',
 'Body',
 'OwnerUserId',
 'AcceptedAnswerId',
 'FavoriteCount',
 'ContentLicense',
 'PostTypeId',
 'Tags',
 'OwnerDisplayName',
 'Title',
 'ViewCount',
 'LastActivityDate',
 'CommunityOwnedDate',
 'CommentCount',
 'LastEditorUserId',
 'LastEditorDisplayName',
 'LastEditDate']

## Escritura del formato CSV

El formato CSV está especificado en el estándar RFC 4180. https://www.ietf.org/rfc/rfc4180.txt. En general se puede utilizar la biblioteca `csv` de Python 3 y vamos a exportar una línea de cabecera con todos los campos. https://docs.python.org/3/library/csv.html.

Tendremos en cuenta que todas las filas tienen que tener las mismas columnas y en el mismo orden dado por `all_attrs`.

In [22]:
import csv

def write_csv(destfile: str, all_attrs: list[str], iterator: Iterator[dict[str,str]]) -> None:
  with open(destfile, 'w') as wf:
    cw = csv.writer(wf)

    # Escribir la línea de cabecera
    cw.writerow(all_attrs)

    # Recorrer el iterador
    for row in iterator:
      row_to_write: list[str] = [row.get(att, '') for att in all_attrs]
      cw.writerow(row_to_write)

In [23]:
write_csv('Posts.csv', all_attrs, generate_elements_from_lines('Posts.xml'))

In [24]:
!head Posts.csv

Id,Score,CreationDate,AnswerCount,ParentId,ClosedDate,Body,OwnerUserId,AcceptedAnswerId,FavoriteCount,ContentLicense,PostTypeId,Tags,OwnerDisplayName,Title,ViewCount,LastActivityDate,CommunityOwnedDate,CommentCount,LastEditorUserId,LastEditorDisplayName,LastEditDate
1,40,2015-10-29T15:56:52.933,1,,,"&lt;p&gt;Estoy creando un servicio usando &lt;em&gt;ASP.NET WebApi&lt;/em&gt;. Quiero añadir soporte para la negociación del tipo de contenido basado en extensiones en el &lt;em&gt;URI&lt;/em&gt;, así que he añadido lo siguiente al código de inicialización del servicio:&lt;/p&gt;&#xA;&#xA;&lt;pre&gt;&lt;code&gt;public static class WebApiConfig&#xA;{&#xA;  public static void Register(HttpConfiguration config)&#xA;  {&#xA;    config.Formatters.JsonFormatter.AddUriPathExtensionMapping(&quot;json&quot;, &quot;application/json&quot;);&#xA;    config.Formatters.XmlFormatter.AddUriPathExtensionMapping(&quot;xml&quot;, &quot;application/xml&quot;);&#xA;  }&#xA;}&#xA;&lt;/code&gt;&lt;/pre&gt;&#xA;&#

## Conversión hacia JSON

El siguiente código convierte el fichero CSV en al formato JSON que podéis ver en: https://www.json.org/json-en.html. El código funciona, pero tiene el problema de que para convertir todo a JSON, se tiene que generar un objeto (diccionario) JSON con **todos** los datos, que se tiene que almacenar en memoria. Esto no es siempre posible. Después veremos otro formato que no tiene este problema.

In [25]:
import json

def csv_to_json(fname_csv: str, fname_json: str, primary_key: str) -> None:
    data_dict: dict[str,dict] = {}

    with open(fname_csv, "r") as f_csv:
        csv_reader = csv.DictReader(f_csv)

        for row in csv_reader:
            key: str = row[primary_key]
            data_dict[key] = row

    with open(fname_json, 'w') as f_json:
        f_json.write(json.dumps(data_dict, indent=4))

In [26]:
fname_csv = 'Posts.csv'
fname_json = 'Posts.json'

csv_to_json(fname_csv, fname_json, 'Id')

In [27]:
!head Posts.json

{
    "1": {
        "Id": "1",
        "Score": "40",
        "CreationDate": "2015-10-29T15:56:52.933",
        "AnswerCount": "1",
        "ParentId": "",
        "ClosedDate": "",
        "Body": "&lt;p&gt;Estoy creando un servicio usando &lt;em&gt;ASP.NET WebApi&lt;/em&gt;. Quiero a\u00f1adir soporte para la negociaci\u00f3n del tipo de contenido basado en extensiones en el &lt;em&gt;URI&lt;/em&gt;, as\u00ed que he a\u00f1adido lo siguiente al c\u00f3digo de inicializaci\u00f3n del servicio:&lt;/p&gt;&#xA;&#xA;&lt;pre&gt;&lt;code&gt;public static class WebApiConfig&#xA;{&#xA;  public static void Register(HttpConfiguration config)&#xA;  {&#xA;    config.Formatters.JsonFormatter.AddUriPathExtensionMapping(&quot;json&quot;, &quot;application/json&quot;);&#xA;    config.Formatters.XmlFormatter.AddUriPathExtensionMapping(&quot;xml&quot;, &quot;application/xml&quot;);&#xA;  }&#xA;}&#xA;&lt;/code&gt;&lt;/pre&gt;&#xA;&#xA;&lt;p&gt;Para que esto funcione necesito crear dos rutas para cada 

Si nos damos cuenta, tenemos el problema de que el valor Id está por duplicado.

Vamos a ver cómo eliminar columnas que no queramos tener.


In [28]:
def csv_to_json2(fname_csv: str, fname_json: str, primary_key: str) -> None:
    data_dict: dict[str, dict] = {}

    with open(fname_csv, "r") as f_csv:
        csv_reader = csv.DictReader(f_csv)

        for rows in csv_reader:
            key: str = rows[primary_key]

            # Borramos los campos que nos interesen.
            del rows[primary_key]

            data_dict[key] = rows

    with open(fname_json, 'w') as f_json:
        f_json.write(json.dumps(data_dict, indent=4))

In [29]:
fname_csv = 'Posts.csv'
fname_json = 'Posts.json'

csv_to_json2(fname_csv, fname_json, 'Id')

In [30]:
!head -n 100 Posts.json

{
    "1": {
        "Score": "40",
        "CreationDate": "2015-10-29T15:56:52.933",
        "AnswerCount": "1",
        "ParentId": "",
        "ClosedDate": "",
        "Body": "&lt;p&gt;Estoy creando un servicio usando &lt;em&gt;ASP.NET WebApi&lt;/em&gt;. Quiero a\u00f1adir soporte para la negociaci\u00f3n del tipo de contenido basado en extensiones en el &lt;em&gt;URI&lt;/em&gt;, as\u00ed que he a\u00f1adido lo siguiente al c\u00f3digo de inicializaci\u00f3n del servicio:&lt;/p&gt;&#xA;&#xA;&lt;pre&gt;&lt;code&gt;public static class WebApiConfig&#xA;{&#xA;  public static void Register(HttpConfiguration config)&#xA;  {&#xA;    config.Formatters.JsonFormatter.AddUriPathExtensionMapping(&quot;json&quot;, &quot;application/json&quot;);&#xA;    config.Formatters.XmlFormatter.AddUriPathExtensionMapping(&quot;xml&quot;, &quot;application/xml&quot;);&#xA;  }&#xA;}&#xA;&lt;/code&gt;&lt;/pre&gt;&#xA;&#xA;&lt;p&gt;Para que esto funcione necesito crear dos rutas para cada acci\u00f3n del con

Al escribir en formato JSON se nos queda un fichero compacto que no podemos dividir.

## JSON Lines

Para evitar el problema anterior (que todo el fichero es un JSON gigante que hay que leer en memoria antes de procesarlo), se creó el formato JSON Lines. En vez de tener un array que incluya a todo el fichero, se obliga a que cada objeto JSON incluido en el fichero esté en su propia línea.

Si algún elemento del JSON contiene un salto de línea, se codifica de alguna forma, como por ejemplo como `'\n'`. De esta forma ya están los datos en CSV, así que la conversión no será problemática.

Más información: https://jsonlines.org.

In [31]:
import json

def csv_to_jsonl(fname_csv, fname_jsonl):
    with open(fname_csv, 'r') as f_csv:
        csv_reader = csv.DictReader(f_csv)

        with open(fname_jsonl, 'w') as f_jsonl:
            for row in csv_reader:
                json_line: str = json.dumps(row)
                f_jsonl.write(json_line)
                f_jsonl.write("\n")

In [32]:
csv_to_jsonl('Posts.csv', 'Posts.jsonl')

In [33]:
!head Posts.jsonl

{"Id": "1", "Score": "40", "CreationDate": "2015-10-29T15:56:52.933", "AnswerCount": "1", "ParentId": "", "ClosedDate": "", "Body": "&lt;p&gt;Estoy creando un servicio usando &lt;em&gt;ASP.NET WebApi&lt;/em&gt;. Quiero a\u00f1adir soporte para la negociaci\u00f3n del tipo de contenido basado en extensiones en el &lt;em&gt;URI&lt;/em&gt;, as\u00ed que he a\u00f1adido lo siguiente al c\u00f3digo de inicializaci\u00f3n del servicio:&lt;/p&gt;&#xA;&#xA;&lt;pre&gt;&lt;code&gt;public static class WebApiConfig&#xA;{&#xA;  public static void Register(HttpConfiguration config)&#xA;  {&#xA;    config.Formatters.JsonFormatter.AddUriPathExtensionMapping(&quot;json&quot;, &quot;application/json&quot;);&#xA;    config.Formatters.XmlFormatter.AddUriPathExtensionMapping(&quot;xml&quot;, &quot;application/xml&quot;);&#xA;  }&#xA;}&#xA;&lt;/code&gt;&lt;/pre&gt;&#xA;&#xA;&lt;p&gt;Para que esto funcione necesito crear dos rutas para cada acci\u00f3n del controlador (estoy usando exclusivamente enrutamient

## Uso de Parquet

![Parquet](https://upload.wikimedia.org/wikipedia/commons/4/47/Apache_Parquet_logo.svg)


El formato Parquet (https://parquet.apache.org) se ha popularizado recientemente con el uso de fuentes de datos en Internet. En general supone una mejora en todos los aspectos con respecto a CSV y en otros con respecto a JSON y JSON lines.

En general, Parquet es un formato de almacenamiento de datos de columnas, que es muy eficiente en términos de espacio y tiempo de acceso. Es un formato binario, pero que se puede leer en muchos lenguajes de programación. Además, permite compresión de datos, lo que lo hace eficiente en tiempo y en espacio.

El formato interno del fichero se describe por encima en la siguiente imagen:

![Parquet](https://camo.githubusercontent.com/d713741348fd88809ec0809de0a9aea7a6358b04f7d2aace673c9286ee290dfb/68747470733a2f2f7261772e6769746875622e636f6d2f6170616368652f706172717565742d666f726d61742f6d61737465722f646f632f696d616765732f46696c654c61796f75742e676966)

El formato Parquet incluye, además de los datos, el esquema de los mismos, lo que hace que se pueda leer sin dar lugar a errores. Esto soluciona el problema que nos encontramos en CSV y JSON, que no incluyen el esquema de los datos.

Es incluso fomentado por el Gobierno de España para publicación de los datos: https://datos.gob.es/es/blog/por-que-deberias-de-usar-ficheros-parquet-si-procesas-muchos-datos.

En Python se puede leer con la biblioteca `pyarrow` (https://arrow.apache.org/docs/python/parquet.html).


In [34]:
%pip install pyarrow

In [35]:
# Write the df dataframe to parquet file
import pandas as pd
import time

start_time: float = time.time()
df_csv: pd.DataFrame = pd.read_csv('Posts.csv')
print("Tiempo de lectura CSV: %s segundos" % (time.time() - start_time))
df_csv.to_parquet('Posts.parquet', compression='snappy')

Tiempo de lectura CSV: 18.396922826766968 segundos


In [36]:
!ls -lh Posts.*

-rw-r--r-- 1 root root 921M Jan 30 11:04 Posts.csv
-rw-r--r-- 1 root root 1.2G Jan 30 11:20 Posts.json
-rw-r--r-- 1 root root 1.1G Jan 30 11:21 Posts.jsonl
-rw-r--r-- 1 root root 359M Jan 30 11:31 Posts.parquet
-rw-r--r-- 1 root root 983M Dec  4  2023 Posts.xml


Se puede ver el contenido del fichero Parquet, pero no es legible, porque está en binario.

In [37]:
!head Posts.parquet

 
 
 
 
 
 
 
 
 
 


In [38]:
start_time: float = time.time()
df_parquet: pd.DataFrame = pd.read_parquet('Posts.parquet')
print("Tiempo de lectura Parquet: %s segundos" % (time.time() - start_time))

Tiempo de lectura Parquet: 5.833038568496704 segundos


Téngase en cuenta que el Parquet guarda la información de tipos de datos, por lo que no es necesario especificarlos.

In [39]:
df_parquet.dtypes

,0
Id,int64
Score,int64
CreationDate,object
AnswerCount,float64
ParentId,float64
ClosedDate,object
Body,object
OwnerUserId,float64
AcceptedAnswerId,float64
FavoriteCount,float64


In [40]:
df_parquet.head()

,Id,Score,CreationDate,AnswerCount,ParentId,ClosedDate,Body,OwnerUserId,AcceptedAnswerId,FavoriteCount,...,Tags,OwnerDisplayName,Title,ViewCount,LastActivityDate,CommunityOwnedDate,CommentCount,LastEditorUserId,LastEditorDisplayName,LastEditDate
0,1,40,2015-10-29T15:56:52.933,1.0,NaN,None,&lt;p&gt;Estoy creando un servicio usando &lt;...,23.0,2.0,NaN,...,&lt;.net&gt;&lt;asp.net-web-api&gt;&lt;asp.net...,None,La creación manual de un alias de ruta con un ...,780.0,2021-03-12T17:49:40.687,None,2,NaN,user13558,2019-07-07T21:36:17.737
1,2,31,2015-10-29T19:14:23.673,NaN,1.0,None,&lt;p&gt;He encontrado la solución.&lt;/p&gt;&...,23.0,NaN,NaN,...,None,None,None,NaN,2015-10-29T19:14:23.673,None,2,NaN,None,None
2,3,20,2015-10-29T23:54:31.947,3.0,NaN,None,"&lt;p&gt;Luego de ver cierto código, me he dad...",21.0,9.0,NaN,...,&lt;delphi&gt;,None,¿Es igual utilizar .AsString que .Text para ob...,1035.0,2015-12-18T20:19:25.133,None,1,20.0,None,2015-12-03T16:24:56.370
3,4,6,2015-10-30T00:45:47.640,NaN,3.0,None,&lt;p&gt;&lt;code&gt;.AsString&lt;/code&gt; de...,24.0,NaN,NaN,...,None,None,None,NaN,2015-12-14T00:58:15.613,None,0,25.0,None,2015-12-14T00:58:15.613
4,5,37,2015-10-30T01:15:27.267,7.0,NaN,None,&lt;p&gt;¿Cuál es la forma más eficiente de se...,24.0,208.0,NaN,...,&lt;c++&gt;&lt;string&gt;,None,¿Cómo separar las palabras que contiene un str...,37867.0,2020-06-30T02:23:26.197,None,1,729.0,None,2016-09-27T17:46:24.900


In [41]:
print("Tamaño del archivo CSV:", df_csv.memory_usage(deep=True).sum(), "bytes")

Tamaño del archivo CSV: 2045314556 bytes


In [42]:
print("Tamaño del archivo Parquet:", df_parquet.memory_usage(deep=True).sum(), "bytes")

Tamaño del archivo Parquet: 1206837975 bytes


Una de las ventajas del formato Parquet es que puedes leer solo las columnas que necesitas, lo cual es útil para trabajar con grandes conjuntos de datos.

In [43]:
df_parquet_subset: pd.DataFrame = pd.read_parquet('Posts.parquet', columns=['PostTypeId', 'CreationDate'])
print(df_parquet_subset.head())

   PostTypeId             CreationDate
0           1  2015-10-29T15:56:52.933
1           2  2015-10-29T19:14:23.673
2           1  2015-10-29T23:54:31.947
3           2  2015-10-30T00:45:47.640
4           1  2015-10-30T01:15:27.267


Se pueden dividir los datos en varios archivos Parquet (por ejemplo, particionados por el tipo de licencia, score, etc.) y luego cargar esos archivos de manera eficiente.

In [44]:
df_parquet.to_parquet('Posts_partitioned/', partition_cols=['ContentLicense'])

In [45]:
!ls -R Posts_partitioned

Posts_partitioned:
'ContentLicense=CC%20BY-SA%203.0'  'ContentLicense=CC%20BY-SA%204.0'

'Posts_partitioned/ContentLicense=CC%20BY-SA%203.0':
1b621770c625411b82e99b0d7d281414-0.parquet

'Posts_partitioned/ContentLicense=CC%20BY-SA%204.0':
1b621770c625411b82e99b0d7d281414-0.parquet


In [47]:
del df_parquet
del df_csv

In [46]:
!tree "Posts_partitioned"

Posts_partitioned
├── ContentLicense=CC%20BY-SA%203.0
│   └── 1b621770c625411b82e99b0d7d281414-0.parquet
└── ContentLicense=CC%20BY-SA%204.0
    └── 1b621770c625411b82e99b0d7d281414-0.parquet

2 directories, 2 files
